<a href="https://colab.research.google.com/github/kcw0331/Environmental-Health-Big-Data-Analysis/blob/main/6%EC%A3%BC%EC%B0%A8/3-making_dataframes_from_api_requests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_02

/content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_02


# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [ ]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [ ]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [ ]:
earthquake_json['metadata']

{'generated': 1604267813000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2020-10-01&endtime=2020-10-31',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.10.3',
 'count': 13706}

Each element in the JSON array `features` is a row of data for our dataframe.

In [ ]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [ ]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1,
  'place': '50 km ENE of Susitna North, Alaska',
  'time': 1604102395919,
  'updated': 1604103325550,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak020dz5f85a',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak020dz5f85a&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 15,
  'net': 'ak',
  'code': '020dz5f85a',
  'ids': ',ak020dz5f85a,',
  'sources': ',ak,',
  'types': ',origin,phase-data,',
  'nst': None,
  'dmin': None,
  'rms': 1.36,
  'gap': None,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.0 - 50 km ENE of Susitna North, Alaska'},
 'geometry': {'type': 'Point', 'coordinates': [-148.9807, 62.3533, 5]},
 'id': 'ak020dz5f85a'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [ ]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]

In [ ]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,alert,status,tsunami,sig,net,code,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.19,"15 km SSE of Sunnyside, Utah",1633651087850,1633706321620,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,None,reviewed,0,22,uu,60462102,",uu60462102,",",uu,",",origin,phase-data,",7.0,0.01108,0.08,194.0,md,earthquake,"M 1.2 - 15 km SSE of Sunnyside, Utah"
1,0.94,"9km NNW of Anza, CA",1633650887410,1633709518540,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,None,reviewed,0,14,ci,40065448,",ci40065448,",",ci,",",focal-mechanism,nearby-cities,origin,phase-da...",26.0,0.05044,0.11,52.0,ml,earthquake,"M 0.9 - 9km NNW of Anza, CA"
2,1.76,"6 km ESE of P?hala, Hawaii",1633650829380,1633651025860,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,None,automatic,0,48,hv,72745172,",hv72745172,",",hv,",",origin,phase-data,",20.0,NaN,0.14,179.0,md,earthquake,"M 1.8 - 6 km ESE of P?hala, Hawaii"
3,2.00,"54 km NNW of Petersville, Alaska",1633650453004,1633651673787,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,None,automatic,0,62,ak,021cvggad3,",ak021cvggad3,",",ak,",",origin,",NaN,NaN,0.28,NaN,ml,earthquake,"M 2.0 - 54 km NNW of Petersville, Alaska"
4,2.42,"9 km SSE of Indios, Puerto Rico",1633650253580,1633651941360,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,None,reviewed,0,90,pr,2021280003,",pr2021280003,",",pr,",",origin,phase-data,",11.0,0.10810,0.10,232.0,md,earthquake,"M 2.4 - 9 km SSE of Indios, Puerto Rico"


### (Optional) Write Data to CSV

In [ ]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>